In [40]:
import pandas as pd
import numpy as np
import h5py
import os

from glob import glob

In [41]:
# This return the maximum value in a given nd.array
def get_max_list(a_list):
    # Converts the array to a dataframe
    df = pd.DataFrame(a_list)
    max_values = df.max()
    return max_values

In [42]:
# Get the RAS Folder and get all .p##.hdf files
ras_folder = r"C:\Users\richmond\AMH Philippines, Inc\PP23.307 Rockwell and Roll - My Documents\Python\amh_pyqgis\flow_extraction\RAS"
glob_key = os.path.join(ras_folder, "*.p*.hdf")
ras_plan = glob(glob_key, recursive=True)

In [43]:
def flow_extract(plan_file):
    ras_hdf_file = plan_file

    # Read the HDF5 File
    f =  h5py.File(ras_hdf_file, 'r')

    # Get the Plan ShortID - this represents
    # the simulation or return period
    short_id_path = '/Plan Data/Plan Information'
    plan_id = f[short_id_path].attrs['Plan ShortID'].decode('utf-8')

    # Get the names of the reference line
    ref_name_path = r'/Geometry/Reference Lines/Attributes'
    ref_names = np.array(f[ref_name_path])
    ref_col_names = [x[0].decode('utf-8') for x in ref_names]

    # Get the max discharge per reference lines
    ref_line_path = r'/Results/Unsteady/Output/Output Blocks/DSS Hydrograph Output/Unsteady Time Series/Reference Lines'
    ref_flow_array = f[ref_line_path]['Flow']
    flow_max_values = np.max(ref_flow_array, axis=0)
    ref_flow = flow_max_values.tolist()

    # Get the Max Velocity per Reference Lines
    ref_vel_array = f[ref_line_path]['Velocity']
    vel_max_values = np.max(ref_vel_array, axis=0)
    ref_velocity = vel_max_values.tolist()

    # Compute the Max Flow Area
    # Given by the Eq. Q = A * V; A = Q / V
    ref_max_flow_area_array = np.divide(
                                ref_flow_array,
                                ref_vel_array,
                                )
    ref_max_flow_area = np.nanmax(ref_max_flow_area_array, axis=0).tolist()

    # Get the Max WSE
    ref_wse_array = f[ref_line_path]['Water Surface']
    ref_wse = np.max(ref_wse_array, axis=0).tolist()

    # Compute the EGL
    # Given by the equation: EGL = WSE + V^2 / 2g (g=9.81 m/s^2)
    g = 9.81 * 2 
    vel_head = np.power(ref_vel_array, 2) / g
    egl_array = ref_wse + vel_head
    ref_egl = np.max(egl_array, axis=0).tolist()

    # Create a DataFrame
    df = pd.DataFrame({
        'Reference Line' : ref_col_names,
        'Plan ID' : plan_id, 
        'Discharge' : ref_flow,
        'Velocity' : ref_velocity,
        'Flow Area': ref_max_flow_area,
        'WSE': ref_wse,
        'EGL': ref_egl
    })

    f.close()

    return df

In [44]:
# Complete columns for report
x= """
This is the complete column names:

column = [
    'Reference Line',
    'Discharge',
    'Flow Velocity',
    'Flow Area',
    'WSE',
    'EGL',
    'Thalweg',
    'LOB',
    'ROB',
]

"""

In [45]:
dfs = []

for plan in ras_plan:
    df_extract = flow_extract(plan)
    dfs.append(df_extract)

df_concat = pd.concat(dfs, ignore_index=True).sort_values(by='Reference Line')

C:\Users\richmond\AppData\Local\Temp\ipykernel_15952\2483269163.py:30: RuntimeWarning: invalid value encountered in divide
  ref_max_flow_area_array = np.divide(
